# Introdução a Web scraping

Organização e Recuperação da Informação
Professor: Wendel Melo


Referência:

[1] Ryan Mitchell, Web scraping com Python. O'Reilly, 2ª edição, 2019.

**Links da aula**
* https://pt.wikipedia.org/wiki/Zeca_Pagodinho
* https://pt.wikipedia.org/wiki/Lista_de_universidades_federais_do_Brasil


**Código útil**

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#Exemplo: programa que visita uma página da wikipedia e recolhe links que apontem para outros artigos da wikipedia.

Para identificar esses links usaremos a estratégia [1]:

1. Os links estão contidos na div com id "bodyContent"
2. Os URL's não contém dois pontos
3. Os URL começam com /wiki/

Note que, para encontrar links com as características 2 e 3, será mais fácil usar expressões regulares.

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

"""Função que recebe uma URL de um artigo da wikipedia e retorna uma lista com links
para outros artigos da wikipedia contigos nessa URL"""
def pegaWikiLinks( urlWiki ):
  html = urlopen( urlWiki )
  bs = BeautifulSoup( html, 'html.parser' )
  divbc = bs.find('div', {'id': 'bodyContent'} ) #pega a div bodyContent
  #TODO: COLOCAR UM COMENTÁRIO AQUI
  expregular = re.compile( '^(/wiki/)((?!:).)*$' )
  links = divbc.find_all( 'a', {'href': expregular}  )
  urlBase = "https://pt.wikipedia.org"
  links = [ urlBase + a['href'] for a in links ]
  return links

javisitados = set()

url = 'https://pt.wikipedia.org/wiki/Zeca_Pagodinho'

filaLinks = []
iter = 0
MAXITERS = 200

while iter < MAXITERS:
  iter += 1
  javisitados.add( url )
  links = pegaWikiLinks( url )

  for link in links:
    if link not in javisitados:
      filaLinks.append(link)

  print(links)

  if len(filaLinks) == 0:
    break

  url = filaLinks.pop(0)  #pega o próximo url da fila

['https://pt.wikipedia.org/wiki/TVZ', 'https://pt.wikipedia.org/wiki/4_de_fevereiro#Nascimentos', 'https://pt.wikipedia.org/wiki/1959', 'https://pt.wikipedia.org/wiki/Rio_de_Janeiro', 'https://pt.wikipedia.org/wiki/Rio_de_Janeiro_(estado)', 'https://pt.wikipedia.org/wiki/Brasileiros', 'https://pt.wikipedia.org/wiki/Canto_(m%C3%BAsica)', 'https://pt.wikipedia.org/wiki/Compositor', 'https://pt.wikipedia.org/wiki/G%C3%AAnero_musical', 'https://pt.wikipedia.org/wiki/Samba', 'https://pt.wikipedia.org/wiki/Partido-alto', 'https://pt.wikipedia.org/wiki/Pagode_(estilo_musical)', 'https://pt.wikipedia.org/wiki/MPB', 'https://pt.wikipedia.org/wiki/Instrumento_musical', 'https://pt.wikipedia.org/wiki/Canto_(m%C3%BAsica)', 'https://pt.wikipedia.org/wiki/Cavaquinho', 'https://pt.wikipedia.org/wiki/Gravadora', 'https://pt.wikipedia.org/wiki/RGE', 'https://pt.wikipedia.org/wiki/BMG', 'https://pt.wikipedia.org/wiki/PolyGram', 'https://pt.wikipedia.org/wiki/Universal_Music', 'https://pt.wikipedia.org/w

# Exemplo: programa que obtém o primeiro parágrafo da wikipedia sobre cada uma das universidades federais do Brasil

Usaremos a estratégia

1. Obter a lista com todas as universidades federais
2. Visitar o link de cada uma delas e obter o primeiro parágrafo

Para obter o título, procuraremos pela primeira tag h1
Para obter o primeiro parágrafo, procuraremos pela primeira tag p após a primeira tag table.

Vamos salvar os dados obtidos em um arquivo dbm. Os arquivos dbm funcionam como se fossem dicionários, isto é, através de pares chave e ítem, com a limitação de arquivos dbm só podem trabalhar com strings.